In [3]:
from arcgis import GIS
import pandas as pd
from datetime import datetime
from time import sleep

In [5]:
gis = GIS(username="username")

Enter password: ········


In [6]:
def getInventaryItems(user):    
    datos = []
    foldersName = [None]+[folder["title"] for folder in user.folders]
    for folder in foldersName:
        items = user.items(folder)
        if folder is None:
            folder = "Root"
        for item in items:
            datos.append([folder, item.owner, item.title, item.name, item.type, item.numViews, item.size/1048576 , 
            datetime.utcfromtimestamp(item.created/1000).strftime('%Y-%m-%d %H:%M:%S'),
            datetime.utcfromtimestamp(item.modified/1000).strftime('%Y-%m-%d %H:%M:%S'), item.id])
    return datos   

In [ ]:
columns = ["Folder","Propietario", "Título", "Nombre", "Tipo", "Número de vistas", 
           "Tamaño (mb)", "Fecha creación", "Fecha modificación", "Item ID"]
with pd.ExcelWriter('Inventario AGOL.xlsx', engine='xlsxwriter') as writer:     
    for user in gis.users.search():
        datos = getInventaryItems(user)
        df = pd.DataFrame(datos, columns=columns)
        df.to_excel(writer, sheet_name=user.fullName)
        del df
        sleep(3)
        
writer.save()
